In [13]:
import numpy as np


def get_mean_structure(path: str, coords_range: tuple|None = None, step_size: int = 1):
    with open(path) as f:
        fl = f.readlines()
    
    cell = np.asarray([_.split() for _ in fl[2:5]], dtype=np.float64)
    n_atom = sum(int(_) for _ in fl[6].split())  # 列表推导式
    n_step = int(fl[-n_atom-1].split()[-1])
    if coords_range is not None:
        if len(coords_range) != 2: raise ValueError('coords_range must be None or 2 element tuple')
        if coords_range[0] < 1: raise ValueError('Invalid lower bound value.')
        if coords_range[0] > coords_range[1]: raise ValueError('lower bound is greater than upper bound.')
        if coords_range[-1] > n_step: coords_range = (coords_range[0], n_step)
    else:
        coords_range = (0, n_step)
    coordinates = fl[7:]  # all coordinates without cell vector & elements.
    configurations = list()
    for i in range(coords_range[0] - 1, coords_range[1], step_size):
        coord = coordinates[(n_atom + 1) * i: (n_atom + 1) * (i + 1)]
        coord = [_.split() for _ in coord[1:]]
        configurations.append(coord)
    configurations = np.asarray(configurations, dtype=np.float64)
    
    return np.mean(configurations @ cell, axis=0)

def diff_coords(X1: np.ndarray, X2: np.ndarray):
    if X1.shape != X2.shape: raise ValueError(f'The shape of X1 {X1.shape} and X2 {X2.shape} does not match.')
    return np.linalg.norm(X1 - X2, )/len(X1)

    
if __name__ == '__main__':
    X1 = get_mean_structure('./PdMoXDATCAR', coords_range=(1000, 6000))
    X2 = get_mean_structure('./PdMoXDATCAR', coords_range=(6000, 10000))
    rms = diff_coords(X1, X2)
    print(rms)

0.06899854450356341
